### Onboarding Waitlist 

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # Suppress SettingWithCopyWarning

In [4]:
onboarding_data = pd.read_csv('onboarding_waitlist.csv', low_memory=False)
registration_data = pd.read_excel('network_register.xlsx', sheet_name='Sheet1')
network_waitlist_data = pd.read_excel('network_waitlist.xlsx', sheet_name='Sheet1')

print(len(onboarding_data))
print(len(registration_data))
print(len(network_waitlist_data))

284
501
268


### Prepare Data and Merge 

In [5]:
network_waitlist_data = network_waitlist_data[~network_waitlist_data["Waitlist No."].isnull()]
print(len(network_waitlist_data))

264


In [6]:
# Strip of leading and tailing white space
onboarding_data["safe_address"] = onboarding_data["safe_address"].str.strip()
registration_data["What is your HOPR safe address?"] = registration_data["What is your HOPR safe address?"].str.strip()
network_waitlist_data["Safe address"] = network_waitlist_data["Safe address"].str.strip()

# Make everything lower case letters
onboarding_data["safe_address"] = [x.lower() for x in onboarding_data["safe_address"]]
registration_data["What is your HOPR safe address?"] = [x.lower() for x in registration_data["What is your HOPR safe address?"]]
network_waitlist_data["Safe address"] = [x.lower() for x in network_waitlist_data["Safe address"]]

# Check for duplicates
dataDup_onboarding = onboarding_data.duplicated(subset=['safe_address'], keep='last')
dataDup_registration = registration_data.duplicated(subset=['What is your HOPR safe address?'], keep='last')

# dataDup.value_counts()
onboarding_data['Duplicate'] = dataDup_onboarding
registration_data['Duplicate'] = dataDup_registration

print(len(registration_data))

501


In [7]:
# Only keep unique values
onboarding_data_01 = onboarding_data.loc[onboarding_data['Duplicate'] == False]
registration_data_01 = registration_data.loc[registration_data['Duplicate'] == False]

print(onboarding_data_01['Duplicate'].value_counts())
print(registration_data_01['Duplicate'].value_counts())

False    284
Name: Duplicate, dtype: int64
False    474
Name: Duplicate, dtype: int64


In [8]:
## Delete people who are already in the waitlist from the registration data

x = list(network_waitlist_data[network_waitlist_data['Eligibility'] == "yes"]["Safe address"])
print(len(x))

registration_data_01['waitlist_ind'] = np.where(registration_data_01["What is your HOPR safe address?"].isin(x), "YES", "No")
registration_data_02 = registration_data_01[registration_data_01['waitlist_ind'] == "No"]
print(len(registration_data_02))

150
324


In [10]:
waitlist = registration_data_02.merge(onboarding_data_01, how='left', left_on='What is your HOPR safe address?'
                                                    , right_on='safe_address')

waitlist = waitlist[['Time', 'deployment_date', 'What is your Node address', 'safe_address', 'deployment_tx_hash', 'wxHOPR_balance', 'nr_nft']]

# rename comumns
waitlist = waitlist.rename(columns={"Time": "registration_time", "What is your Node address": "node_address"})
print(len(waitlist))

# exlude non elidgible nodes
waitlist_01 = waitlist[~waitlist['safe_address'].isnull()]
print(len(waitlist_01))
# display(waitlist_01.head())

324
133


### Delete non-elidgible nodes 

In [11]:
# Not enough stake
waitlist_02 = waitlist_01[waitlist_01['wxHOPR_balance'] >= 10000]
print(len(waitlist_02))
print(waitlist_02.dtypes)

122
registration_time      object
deployment_date        object
node_address           object
safe_address           object
deployment_tx_hash     object
wxHOPR_balance        float64
nr_nft                 object
dtype: object


In [12]:
waitlist_02.loc[:, 'nr_nft'] = waitlist_02['nr_nft'].astype(bool)
print(waitlist_02['nr_nft'].value_counts())

True     102
False     20
Name: nr_nft, dtype: int64


/tmp/ipykernel_29643/2374682768.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  waitlist_02.loc[:, 'nr_nft'] = waitlist_02['nr_nft'].astype(bool)


In [13]:
# Users with a stake >= 30000 are eligible regardless of NR NFT
waitlist_03 = waitlist_02.copy()
waitlist_03.loc[:, 'eligible'] = np.where(waitlist_02.loc[:,'wxHOPR_balance'] >= 30000, True, False)

# Users with a stake < 30000 are eligible if they have a network registry NFT
waitlist_04 = waitlist_03.copy()
waitlist_04.loc[:, 'eligible'] = np.where((waitlist_03.loc[:,'wxHOPR_balance'] < 30000) & (waitlist_03.loc[:,'nr_nft'] == True), True, waitlist_03.loc[:, 'eligible'])

print(waitlist_04.loc[:, 'eligible'].value_counts())


True    122
Name: eligible, dtype: int64


In [14]:
waitlist_05 = waitlist_04[waitlist_04.loc[:,'eligible'] == True]
print(len(waitlist_05))

122


In [15]:
waitlist_05["node_address"] = waitlist_05["node_address"].str.strip()

In [16]:
# delete users that registered with their peer_id
valid_address = waitlist_05['node_address'].str.startswith('0x')
waitlist_06 = waitlist_05[valid_address]
print(len(waitlist_06))

119


### Sort Users 

In [17]:
nr_waitlist = waitlist_06[waitlist_06['nr_nft'] == True]
stake_waitlist =waitlist_06[waitlist_06['nr_nft'] == False]

print(len(nr_waitlist))
print(len(stake_waitlist))

99
20


In [18]:
nr_waitlist_01 = nr_waitlist.sort_values(by="deployment_date", ascending=True).reset_index(drop=True)
print(len(nr_waitlist_01))
# display(nr_waitlist_01.head())

99


In [20]:
stake_waitlist_01 = stake_waitlist.sort_values(by="wxHOPR_balance", ascending=False).reset_index(drop=True)
print(len(stake_waitlist))
# display(stake_waitlist_01.head())

20


In [22]:
# Assuming you have two dataframes, nr_waitlist_01 and stake_waitlist_01
nr_chunk_size = 20
stake_chunk_size = 10

nr_index = 0  # Starting index for nr_waitlist_01
stake_index = 0  # Starting index for stake_waitlist_01

final_chunks = []  # List to store concatenated chunks

while nr_index < len(nr_waitlist_01) or stake_index < len(stake_waitlist_01):
    # Add a chunk of nr if there are more rows available
    if nr_index < len(nr_waitlist_01):
        nr_chunk = nr_waitlist_01.iloc[nr_index:nr_index + nr_chunk_size]
        final_chunks.append(nr_chunk)
        nr_index += nr_chunk_size

    # Add a chunk of stake if there are more rows available
    if stake_index < len(stake_waitlist_01):
        stake_chunk = stake_waitlist_01.iloc[stake_index:stake_index + stake_chunk_size]
        final_chunks.append(stake_chunk)
        stake_index += stake_chunk_size

# Concatenate the final chunks row-wise
final_waitlist = pd.concat(final_chunks, ignore_index=True)

# display(final_waitlist.head())
print(len(final_waitlist))

119


In [23]:
### sanity checks

# check users who already got access are not in the waitlist
print(pd.Series(final_waitlist['safe_address']).isin(list(network_waitlist_data[network_waitlist_data['Eligibility'] == "yes"]["Safe address"])).value_counts())

# check that all users who were already in the waitlist are indeed in the waitlist
print(pd.Series(final_waitlist['safe_address']).isin(list(network_waitlist_data[network_waitlist_data['Eligibility'].isnull()]["Safe address"])).value_counts())

# check that no duplicates are in the waitlist
print((final_waitlist.duplicated(subset=['safe_address'], keep='last')).value_counts())

False    119
Name: safe_address, dtype: int64
True     114
False      5
Name: safe_address, dtype: int64
False    119
dtype: int64


### Save final waitlist 

In [24]:
final_waitlist.to_excel('final_waitlist.xlsx', index=False)